# Clean Time Logs

In [4]:
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp

from empkins_micro.time_logs import process_time_log

from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [5]:
base_path = Path("../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject")
subject_dirs = bp.utils.file_handling.get_subject_dirs(data_path, "VP_*")

subject_dirs

[PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_01'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_02'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_03'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_04'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_05'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_06'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_07'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_08'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_09')]

In [52]:
for subject_dir in tqdm(subject_dirs):
    subject_id = subject_dir.name
    for condition in ["control", "stress"]:
        timelog_dir = subject_dir.joinpath("{}/time_log".format(condition))
        cleaned_dir = timelog_dir.joinpath("cleaned")
        timelog = process_time_log(list(cleaned_dir.glob("*.csv"))[0])
        timelog.columns = ["_".join(col) for col in timelog.columns]
        
        timelog = timelog.apply(lambda series: bp.utils.time.timedelta_to_time(bp.utils.time.get_time_from_date(series)))

        processed_dir = timelog_dir.joinpath("processed")
        processed_dir.mkdir(exist_ok=True)
        timelog.to_csv(processed_dir.joinpath("time_log_{}.csv".format(subject_id)))

  0%|          | 0/6 [00:00<?, ?it/s]